In [1]:
import json
from flask import Flask,request
from flask import jsonify
from flask_cors import cross_origin
import tensorflow.keras as keras

import string
import re

from nltk.corpus import stopwords
from nltk import word_tokenize,pos_tag
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet as wn
from collections import defaultdict
from tqdm import tqdm

eng_sw=stopwords.words('english')

tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
  
lemmatizer = WordNetLemmatizer()

with open('./logs/tokenizer.json','r') as f:
    tokenizer_json=json.load(f)

tokenizer=keras.preprocessing.text.tokenizer_from_json(tokenizer_json)
model=keras.models.load_model('./logs/model.h5')

In [2]:
def give_data(sent):
    #punctuation removal
    sent=re.sub('['+string.punctuation+']',' ',sent)
    #tokenize
    tokens=word_tokenize(sent)
    #lower case convertion & POS tagging
    tokens_pos=pos_tag([w.lower() for w in tokens])
    #stopwords removal & only words & len(word)>2 & lemmatization
    major_sent=[lemmatizer.lemmatize(w,tag_map[pos[0]]) for w,pos in tokens_pos if( (w not in eng_sw) 
                                                                                      and (w.isalpha()) 
                                                                                      and (len(w)>2))]
    major_sent=' '.join(major_sent)

    pad_embed_sent=\
    keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([major_sent]),maxlen=500)

    data = [
      {
        'x': ['Negative', 'Positive'],
        'y': [float(e) for e in model.predict(pad_embed_sent)[0]],
        "marker":{"color": ['rgba(255,0,0,0.5)', 'rgba(0,255,0,0.5)']},
        'type': 'bar'
      }
    ];
    return data

In [3]:
app = Flask(__name__)

@app.route('/',methods=['GET'])
@cross_origin()
def hello_world():
    review = request.args.get('review')
    data=give_data(review)
    return jsonify(data)

In [4]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Dec/2020 00:02:18] "GET /?review=i%20love%20it HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2020 00:02:19] "GET /?review=i%20love%20it HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2020 00:02:28] "GET /?review=i%20love%20it HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2020 00:02:39] "GET /?review=i%20love%20it%20very%20much HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2020 00:02:54] "GET /?review=i%20love%20it%20very%20much%20and%20also%20hate%20it%20at%20the%20same%20time HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2020 00:03:54] "GET /?review=i%20love%20it%20as%20much%20as%20i%20hate%20it HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2020 00:05:23] "GET /?review=title:%20%27Least%20Used%20Feature%27 HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2020 00:05:54] "GET /?review=title:%20%27Least%20Used%20Feature%27 HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2020 00:07:25] "GET /?review=title:%20%27Least%20Used%20Feature%27 HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2020 0

In [5]:
#NICE